In [25]:
"""
Data Generation for Extended Kalman Filter and Classification Test
    DT: Target Data
    DO: Observer Data
    
    Various Function that generates data for: 
        optimization, monte carlo simulations, etc.
"""

"""
Define Various Functions: 
    
    Obs_Targ_Data1: Generates Data based on the # of points (N)
    Obs_Targ_Data2: Generates Data based on the desired final time

"""

'\nDefine Various Functions: \n    \n    Obs_Targ_Data1: Generates Data based on the # of points (N)\n    Obs_Targ_Data2: Generates Data based on the desired final time\n\n'

In [14]:
import numpy as np
import scipy as sp
from scipy.stats import qmc
from sklearn.neighbors import NearestNeighbors
from scipy.integrate import solve_ivp
from ipynb.fs.full.funcCR3BP import CR3BP

In [15]:
def Obs_Targ_Data1(XObs, Xtarg, mu, tsteps, N):
    """
    # Build the Observer and Target Matrix
    
    Parameters # All UNITS are normalized
    ----------
    XObs = States of the Observer
    Xtarg = States of the Target
    mu = mass parameter
    tsteps = desired time steps
    N = # of points
    
    Returns = DT, DO [Target and Observer Matrix]
    -------
    """
    # Time Index
    tend = tsteps*N
    ti = np.linspace(0,tend,N+1)
    
    # Generate Observer Data
    sol_obs = solve_ivp(CR3BP, [0, tend], XObs, t_eval = ti, args = (mu,), rtol = 1E-13, atol = 1e-13)
    
    # Generate Target Data
    sol_targ = solve_ivp(CR3BP, [0, tend], Xtarg, t_eval = ti, args = (mu,), rtol = 1E-13, atol = 1e-13)
    
    DO = sol_obs.y[0:3,:]
    DT = sol_targ.y
    
    return DO, DT

In [16]:
def Obs_Targ_Data2(XObs, Xtarg, mu, tsteps, tend):
    """
    Parameters # All UNITS are normalized
    ----------
    XObs = States of the Observer
    Xtarg = States of the Target
    mu = mass parameter
    tsteps = desired time steps
    tend = desired end time
    
    Returns = DT, DO [Target and Observer Matrix]
    -------
    """
    # Time Index
    ti = np.arange(0, tend, tsteps)
    
    # Generate Observer Data
    sol_obs = solve_ivp(CR3BP, [0, tend], XObs, t_eval = ti, args = (mu,), rtol = 1E-13, atol = 1E-13)
    
    # Generate Target Data
    sol_targ = solve_ivp(CR3BP, [0, tend], Xtarg, t_eval = ti, args = (mu,), rtol = 1E-13, atol = 1E-13)
    
    DO = sol_obs.y[0:3,:] # Position for Observer
    DT = sol_targ.y # Utilize Target Data to calculate Error
    
    return DO, DT

In [17]:
def randomSampleIC(r, r_pert, v_pert, DU, TU, mu, N):
    """
    # randomSample1: Provides the Inital Conditions, sampling a design space
    # The design space is points on the sphere of influence around the target body
    
    Parameters
    ----------
    r = radius of target
    r_pert = Define the radius of the sphere in km
    v_pert = Define random perturbation of velocity
    DU = distance unit of the system
    TU = time unit of the system
    mu = mass parameter
    N = amount of samples
    
    Default Body: Moon (CR3BP)
    
    Returns: Inital Conditions for randomly generated samples X_IC
    -------
    """
    # Define the Sphere of Influence around the Moon
    s_norm = (r + r_pert)/DU # Normalized Length Units
    v_norm = v_pert*(DU/TU) # Normalized Velocity Units
    c = np.array([[1-mu],[0], [0]])  # Center of the target body
    
    X_IC = np.zeros([6,N])
    ran_num = np.random.normal(0,0.1, size = (3,N)) # Generate Gaussian Normal Distribution
    
    x,y,z = ran_num[:,:]
    eq1 = 1/np.sqrt(x**2 + y**2 + z**2) 
    
    for i in range(N):
        X_IC[0:3,i] = eq1[i]*np.array([x[i],y[i],z[i]])*s_norm # Points on the Sphere
        X_IC[3:,i] = np.random.rand(3,) * v_norm # Random velocity
    
    X_IC[0,:] = X_IC[0,:] + c[0] # Move the origin
    return X_IC

In [18]:
def LHC_Sample(N, rmin, rmax, data):
    """
    ### Apply LatinHyperCube Sampling of Data to create a template for knn search to identify associated data in bins
        Use Primarly for Observer Orbits
        Will not work if data spread is to wide aka data are outside of the respective bins
    Parameters
    ----------
    N = # Amount of Samples [Single Integer]
    rmin = lower bound of data
    rmax = upper bound of data
    data = data that we are sampling
    Returns: 
    -------
    """
    #if rmin >= rmax or N > np.size(data,1):
        #raise("Error: min is greater or equal to max")
        
    LHC = qmc.LatinHypercube(d = 1)
    sLHC = LHC.random(N)
    sLHC = qmc.scale(sLHC, rmin, rmax)
    
    Jdata = data[:,7].reshape(-1,1) # Jacobi Constant Data
    knn = NearestNeighbors(n_neighbors = 1).fit(Jdata)
    idx = knn.kneighbors(sLHC, return_distance = False) # KNN idx
    
    idx = np.unique(idx[:]) # Second Row which house the index closest to the data using LHS
    # Also remove duplicates
    
    return data[idx,:]